# Drug classes: 
I consider two drugs to be in the same class if they have the same moa profile. By studying classes, there could be fruitful insights that could help with cross validation and prediction. Although one could just consider the scored targets, I think considering all targets will be the best. 

### Class naming convention: 
I will name a class by concatenating the names of the moas present in that class. To delineate scored moas from nonscored moas, I will prepend scored moas with ++ and nonscored moas with --, such as the class "++dna_inhibitor--rna_synthesis_inhibitor", which contains an moa from both scored and nonscored moas. 

This is just a starter, and you should go much deeper to see if you can further partition the training set in ways that will hopefully lead to a proper cross validation strategy and better understanding of the feature distribution and the correlation between features and target variables.

In [ ]:
import pandas as pd 
import numpy as np 
import os
################ Directory path #################
# This is so we can easily use our code in 
# different environments by only changing these 
# global variables. 
DATA_RAW_PATH = '../input/lish-moa/'
DATA_INTERIM_PATH = ''

In [ ]:
########################### Read data ###############################
train_features = pd.read_csv(f'{DATA_RAW_PATH}train_features.csv')
test_features = pd.read_csv(f'{DATA_RAW_PATH}test_features.csv')
train_targets_scored = pd.read_csv(f'{DATA_RAW_PATH}train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(f'{DATA_RAW_PATH}train_targets_nonscored.csv')

############### All training data in one dataframe ###################
train_df = train_features.set_index('sig_id')\
    .join(train_targets_scored.set_index('sig_id'))\
    .join(train_targets_nonscored.set_index('sig_id')).reset_index()

################### Feature and target lists ########################
feature_cols = train_features.columns[1:].tolist()
target_scored_cols = train_targets_scored.columns[1:].tolist()
target_nonscored_cols = train_targets_nonscored.columns[1:].tolist()
target_all_cols = target_scored_cols + target_nonscored_cols

In [ ]:
######################## Moa_classes ################################
# All rows that have identical moa activations are considered 
# to be in the same class. We will add class info based on 
# scored_targets, nonscored_targets, and all_targets. I hope that 
# this can help me to classify drugs and make a proper cv strategy. 
# Let's add columns that could be useful. 
train_df['num_scored_targets'] = train_df[target_scored_cols].sum(axis=1)
train_df['num_nonscored_targets'] = train_df[target_nonscored_cols].sum(axis=1)
train_df['num_all_targets'] = train_df['num_scored_targets'] + train_df['num_nonscored_targets']

# Now we will partition the rows into 3 different class groupings
################ 1. Scored target moa_class ###################
df_ = train_df[target_scored_cols]
for col in df_.columns: 
    df_[col] = df_[col].map({1: '++' + col, 0: ''})
train_df['scored_moa_class'] = df_.agg(('').join, axis=1)

################# 2. Nonscored target moa_class ################
df_ = train_df[target_nonscored_cols]
for col in df_.columns: 
    df_[col] = df_[col].map({1: '--' + col, 0: ''})
train_df['nonscored_moa_class'] = df_.agg(('').join, axis=1)

################# 3. All target moa_class ######################
# This will be the most important one to me since I believe 
# it will help the most with creating drug_ids. 
train_df['moa_class'] = train_df['scored_moa_class'] + train_df['nonscored_moa_class']
train_df['moa_class'] = train_df.moa_class.replace('', 'NO_MOA')

##################### No target moa_class ###################
train_df.loc[train_df.cp_type == 'ctl_vehicle', ['moa_class', 'scored_moa_class', 'nonscored_moa_class']] = 'ctl_vehicle'

############# How many rows in moa_class ####################
train_df['rows_in_moa_class'] = train_df.groupby('moa_class')['sig_id'].transform('count')

############# Save new columns to output ####################
class_cols = [col for col in train_df.columns if col not in feature_cols + target_scored_cols + target_nonscored_cols]
train_df[class_cols].to_csv(os.path.join(DATA_INTERIM_PATH, 'class_cols.csv'))

In [ ]:
################### moa_class info ########################
# Lets make a dataframe with all the class info for easy 
# reference later. 
class_df = train_df.drop_duplicates('moa_class')
class_df = class_df[['moa_class', 'rows_in_moa_class', 'num_all_targets',
               'num_scored_targets', 'num_nonscored_targets',
               'scored_moa_class', 'nonscored_moa_class',]].reset_index(drop=True)\
               .sort_values('rows_in_moa_class', ascending=False).reset_index()

################## Save for later ##########################
class_df.to_csv(os.path.join(DATA_INTERIM_PATH, 'class.csv'))
class_df.head(20)

In [ ]:
print(f'There are {class_df.shape[0]} different moa_classes')
print(f'There are {class_df[class_df.rows_in_moa_class <= 6].shape[0]} moa_classes with 6 or less rows')
print(f'There are {class_df[class_df.rows_in_moa_class <= 12].shape[0]} moa_classes with 12 or less rows')
print(f'There are {class_df[class_df.rows_in_moa_class <= 18].shape[0]} moa_classes with 18 or less rows')
print(f'There are {class_df[class_df.rows_in_moa_class <= 24].shape[0]} moa_classes with 24 or less rows')
print(f'There are {class_df[class_df.rows_in_moa_class <= 30].shape[0]} moa_classes with 30 or less rows')
print(f'There are {class_df[class_df.rows_in_moa_class <= 36].shape[0]} moa_classes with 36 or less rows')
print(f'There are {class_df[class_df.rows_in_moa_class <= 42].shape[0]} moa_classes with 42 or less rows')
print(f'There are {class_df[class_df.rows_in_moa_class <= 100].shape[0]} moa_classes with 100 or less rows')
print(f'There are {class_df[class_df.rows_in_moa_class <= 200].shape[0]} moa_classes with 200 or less rows')
print(f'There are {class_df[class_df.rows_in_moa_class <= 300].shape[0]} moa_classes with 300 or less rows')
print(f'There are {class_df[class_df.rows_in_moa_class <= 400].shape[0]} moa_classes with 400 or less rows')

# Visualizing gene and cell data for the first 25 most common moa_classes 
I will simply group rows by moa_class, and then get stats for each gene and cell, but we could (should) group by cp_time and cp_dose to gain more interesting insights, as well as plotting mutliple rows on the same graph. One interesting thing I notice going through these is that sometimes the gene values range between -10 and 10 and sometimes it looks like they go between -1 and 1. Perhaps I need to go deeper to figure out whats going on here.

Make sure to hover over interesting data points to see the gene/cells distribution. I figure if a gene has a mean far from zero, and a narrow range of values or small standard devieation, it should be a good predictor of the moa_class.

In [ ]:
import plotly.express as px
def plot_mean_of_moa_class(moa_class): 
    
    # Data filtering
    title = moa_class + ': ' + \
        str(class_df.loc[class_df.moa_class == moa_class, 'rows_in_moa_class'].tolist()[0]) + ' rows'
    mask = train_df.moa_class == moa_class
    df_ = train_df[feature_cols][mask].iloc[:, 3:]
    stats = df_.describe().T.reset_index().rename({'index': 'gene/cell'}, axis=1)
    
    # Plotting
    data_frame = stats
    x = 'gene/cell'
    y = 'mean'
    hover_data = ['gene/cell', 'min', '25%', '50%', '75%', 'max', 'mean', 'std']
    color = 'std'
    fig = px.scatter(data_frame=data_frame, x=x,y=y, title=title, color=color, hover_data=hover_data)
    fig.show()
    
    
for moa_class in class_df.moa_class.tolist()[:25]:
    plot_mean_of_moa_class(moa_class)